In [1]:
import sys
from pathlib import Path

In [2]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [3]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint

In [4]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    print(str(query))


SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1


In [7]:
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)  
    pages = ['/about', '/contact', '/pages', '/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(
            bucket.label("bucket"),
            EventModel.page,
            func.count().label("event_count"),
        )
        .where(
            EventModel.page.in_(pages),
            EventModel.time >= start,
            EventModel.time <= finish,
        )
        .group_by(
                bucket, 
                EventModel.page
                  )
        .order_by(
            bucket,
            EventModel.page,
            )
    )
    
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    results = session.exec(query).all()
    # pprint(results)

SELECT time_bucket('1 day'::interval, eventmodel.time) AS bucket, eventmodel.page, count(*) AS event_count 
FROM eventmodel 
WHERE eventmodel.page IN ('/about', '/contact', '/pages', '/pricing') AND eventmodel.time >= '2025-07-22 09:30:08.854471+00:00' AND eventmodel.time <= '2025-07-22 11:30:08.854483+00:00' GROUP BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page
